In [ ]:
from fitparse import FitFile

In [ ]:
fitfile = FitFile('./2017-11-03-15-58-49.fit')

In [ ]:
import datetime

the_epoch = datetime.datetime.utcfromtimestamp(0)    

def timestamp_to_epoch(dt):
    return int((dt - the_epoch).total_seconds())


In [ ]:
# Get all data messages that are of type record

hrs = []
tss = []
raw_tss = []
power = []

rec_count = 0
baseline_epoch = 0
baseline_ts = datetime.datetime.now() 
normalized_basetime = datetime.datetime.strptime('24052010', '%d%m%Y')
for record in fitfile.get_messages('record'):
    rec_count += 1
#    if rec_count > 5:
#        break
    print 'record {}'.format(rec_count)

    # Go through all the data entries in this record
    current_epoch = -1
    current_hr = -1
    current_power = -1
    raw_ts = datetime.datetime.now() 
    
    for record_data in record:
        
        if rec_count == 1 and record_data.name == 'timestamp':
            print 'setting baseline epoch from {}'.format(record_data.value)
            baseline_epoch = timestamp_to_epoch(record_data.value)
            baseline_ts = record_data.value
#            print 'baseline epoch be {}'.format(baseline_epoch)

        # Print the records name and value (and units if it has any)
#        if record_data.units:
#            print " * %s: %s %s" % (
#                record_data.name, record_data.value, record_data.units,
#            )
#        else:
#            print " * %s: %s" % (record_data.name, record_data.value)

        if record_data.name == 'power':
            current_power = record_data.value
            
        if record_data.name == 'heart_rate':
            current_hr = record_data.value
                 
        if record_data.name == 'timestamp':    
            current_epoch = timestamp_to_epoch(record_data.value) - baseline_epoch
            raw_ts = normalized_basetime + (record_data.value - baseline_ts)
#            raw_ts = record_data.value
#            print 'current epoch {}'.format(current_epoch)
            
        if current_epoch != -1 and current_hr != -1:
#            print 'i has a tuple ({},{})'.format(current_epoch, current_hr)
            tss.append(current_epoch)
            hrs.append(current_hr)
            raw_tss.append(raw_ts)
            power.append(current_power)
    print

print tss
print hrs
print raw_tss

In [ ]:
# See http://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#import matplotlib.dates
#times = matplotlib.dates.date2num(raw_tss)
import matplotlib.dates as mdates

with plt.xkcd():

    plt.figure(figsize=(15, 8))
    plt.title('heart rate')
    plt.ylabel("BPM")
    myFmt = mdates.DateFormatter('%H:%M')
    plt.gca().xaxis.set_major_formatter(myFmt)
    plt.plot(raw_tss,hrs)
    
plt.show()

In [ ]:
# Get laps
middle = raw_tss[len(raw_tss)/2]
print type(middle)
print middle

messages = fitfile.messages

laps = []
for msg in messages:
    if msg.name == 'lap':
        laps.append(msg)
        
start_times = map(lambda x:x.get_value('start_time'), laps)
print start_times

raw_start_times = map(lambda x:normalized_basetime + (x - baseline_ts), start_times)
print raw_start_times

In [ ]:
import matplotlib.dates as mdates

plt.figure(figsize=(15, 8))
#plt.title('heart rate')
#plt.ylabel("BPM")
myFmt = mdates.DateFormatter('%H:%M')

#plt.plot(raw_tss,hrs)

plt.figure(1)
plt.subplot(211)
plt.title('heart rate (bpm)')
plt.gca().xaxis.set_major_formatter(myFmt)

for st in raw_start_times:
    plt.axvline(x=st,linestyle='dashed',color='k')

#plt.axvline(x=middle)
#plt.axvline(x=raw_start_times[1])
plt.plot(raw_tss,hrs)

plt.subplot(212)
plt.gca().xaxis.set_major_formatter(myFmt)
for st in raw_start_times:
    plt.axvline(x=st, linestyle='dashed',color='k'
               )
plt.plot(raw_tss,power)
plt.title('power (watts)')
plt.show()